In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


# Define the path to your validation dataset
validation_data_dir = 'Devnagari\Test'

# Set up an ImageDataGenerator to load and preprocess images
validation_datagen = ImageDataGenerator(rescale=1./255)

# Specify image size and batch size
img_width, img_height = 224, 224
batch_size = 32

# Create a generator for loading validation images from directories
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',  # Change if using different types of labels
    shuffle=False  # Set to False to maintain order
)

# Extract validation data and labels
X_val = []
y_val = []

num_validation_samples = len(validation_generator.filenames)
num_classes = len(validation_generator.class_indices)

# Load images and labels into lists
for i in range(num_validation_samples):
    img, label = validation_generator.next()
    X_val.append(img)
    y_val.append(label)

# Convert lists to numpy arrays
X_val = np.vstack(X_val)
y_val = np.vstack(y_val)


Found 13800 images belonging to 46 classes.


In [ ]:
import os
from keras.models import load_model

# Path to the directory containing your Keras models
models_directory = 'Models/Models'

# List all the model files in the directory
model_files = os.listdir(models_directory)

# Filter only .h5 files
model_files = [file for file in model_files if file.endswith('.h5')]

# Load models from the directory
models_list = []
for model_file in model_files:
    model_path = os.path.join(models_directory, model_file)
    model = load_model(model_path)
    models_list.append(model)

In [ ]:

from sklearn.model_selection import train_test_split

# Assuming you have a list of trained Keras models stored in `models_list`
# and a validation dataset `X_val`, `y_val` for evaluating model performance

# Split the validation dataset further for calculating the ensemble weights
X_val_ens, X_val_eval, y_val_ens, y_val_eval = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

# Initialize weights for each model in the ensemble
initial_weights = np.ones(len(models_list)) / len(models_list)

# Function to calculate the ensemble predictions
def ensemble_predict(models, weights, X):
    predictions = np.zeros((len(X), len(models)))
    for i, model in enumerate(models):
        predictions[:, i] = model.predict(X).ravel()
    return np.dot(predictions, weights)

# Function to calculate the validation loss/error for given ensemble weights
def calculate_loss(models, weights, X, y):
    y_pred = ensemble_predict(models, weights, X)
    return np.mean(np.square(y - y_pred))  # MSE as loss metric

# Function to optimize ensemble weights
def optimize_weights(models, initial_weights, X, y, num_iterations=100, learning_rate=0.1):
    weights = initial_weights.copy()
    best_weights = weights.copy()
    best_loss = calculate_loss(models, weights, X, y)

    for _ in range(num_iterations):
        gradient = np.zeros(len(models))
        for i in range(len(models)):
            perturbed_weights = weights.copy()
            perturbed_weights[i] += 0.01  # small perturbation
            loss_up = calculate_loss(models, perturbed_weights, X, y)
            gradient[i] = (loss_up - best_loss) / 0.01  # calculate gradient

        weights -= learning_rate * gradient
        loss = calculate_loss(models, weights, X, y)

        if loss < best_loss:
            best_loss = loss
            best_weights = weights.copy()

    return best_weights

# Find the best weights for the ensemble
best_weights = optimize_weights(models_list, initial_weights, X_val_ens, y_val_ens)

print("Best weights for ensemble models:", best_weights)